<a href="https://colab.research.google.com/github/MarioAvolio/FoodX-251-Classification/blob/main/FoodX_251_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Performing data augmentation on a batch of images and the need for collate_fn

Define the Dataset class, which takes the input images, their classes, and
the augmentation object as initializers:

In [ ]:
!pip install -q torch_snippets
from torch_snippets import *
from torchvision.datasets import MNIST
from torchvision import transforms
device = 'cuda' if torch.cuda.is_available() else 'cpu'

!pip install torch_summary
from torchsummary import summary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from torch.utils.data import Dataset, DataLoader
import PIL
class FoodDataset(Dataset):
  def __init__(self, x, y, aug=None):
    self.y = y
    self.x = x 
    self.aug = aug
    self.img_size=(256, 256)
    self.transform = transforms.Compose([transforms.Resize(self.img_size),
                                      transforms.CenterCrop(self.img_size),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.5], [0.5]),
                                      transforms.Lambda(lambda x: x.to(device))])
  
  def __getitem__(self, ix):
    x, y = self.x[ix], self.y[ix]
    return x, y

  def __len__(self): return len(self.x)

  # In general, we leverage the collate_fn method when we have to
  # perform heavy computations. This is because performing such
  # computations on a batch of images in one go is faster than doing it
  # one image at a time.

  # Define collate_fn, which takes the batch of data as input:
  def collate_fn(self, batch):
    # Separate the batch of images and their classes into two different variables
    paths, classes = list(zip(*batch))
    
    # read images
    ims = [Image.open(path) for path in paths]

    # Specify that augmentation must be done if the augmentation object is
    # provided. This is useful is we need to perform augmentation on
    # training data but not on validation data
    if self.aug: ims=self.aug.augment_images(images=ims)

    # Create tensors of images, along with scaling data, by dividing the image shape by 255
    ims = torch.tensor(ims)[:,None,:,:].to(device)/255.
    classes = torch.tensor(classes).to(device)
    return ims, classes

In [ ]:
tr_images = TRAIN_PATH_LOCAL + noise_balanced_dataset.NAME.to_numpy()
tr_images 

array(['/content/train_set/train_050876.jpg',
       '/content/train_set/train_091456.jpg',
       '/content/train_set/train_070941.jpg', ...,
       '/content/train_set/train_004212.jpg',
       '/content/train_set/train_036549.jpg',
       '/content/train_set/train_086447.jpg'], dtype=object)

In [ ]:
tr_targets = noise_balanced_dataset.TYPE.to_numpy()
tr_targets

array([149, 206, 209, ..., 100, 127, 154])

Define the data augmentation pipeline:

In [ ]:
from imgaug import augmenters as iaa
import random

def get_random_scale():
  return random.uniform(0.5, 1.5)

def get_random_translation():
  return {'x':random.randint(-50,50),'y':random.randint(-50,50)}

aug = iaa.Sequential([
iaa.Affine(rotate=(0,360), translate_px=get_random_translation(), scale=get_random_scale(), fit_output=True, mode="edge"),
iaa.SaltAndPepper(0.2),
iaa.GaussianBlur(sigma=1),
# iaa.LinearContrast(0.5),
# iaa.Multiply(0.5),
])

In [ ]:
train = FoodDataset(tr_images, tr_targets, aug=aug)

Next, we define the DataLoader, along with the object's
collate_fn method, as follows:

In [ ]:
trn_dl = DataLoader(train, batch_size=64, collate_fn=train.collate_fn,shuffle=True)